In [1]:
from struct import pack
import serial
import time

In [2]:
#用于检验传输信号的正确性
def sum_byts(data):
    #获取累加校验的结果
    #传入字节串，返回字节串
    #从第二位累加至倒二位，最后一位将被替换为累加的结果
    s=0
    for i in range(len(data)-2):#对接收到的字节串进行遍历累加求和，扣除第一个和第二个字节
        s+=data[i+1]
    s=s & 0xFF #用于将一个整数变量 s 的值限制在8个字节
    data=data[:-1]+pack('B',s) #去掉原字节串的最后一位，并替换为累加结果
    return data


In [3]:
#调整串口
#初始化串口
port="COM4"
byte=9600
se=serial.Serial(port,byte)
se.flushInput()#清空串行端口的输入缓冲区
#握手
data_hello=[0xFE,0xA0,0x00,0x00,0x00,0xA0]#编写握手的指令
se.write(bytes(data_hello))
recv=se.read(6)
if recv[2]==0:
   print("执行握手程序完成")
else:
   print("握手程序执行错误")
time.sleep(2)


AttributeError: module 'serial' has no attribute 'Serial'

In [ ]:
#读取仪器设置信息
se.flushInput()
data_set=[0xFE,0xA1,0x00,0x00,0x00,0xA1]
se.write(bytes(data_set))
recv_set=se.read(11)
print(recv_set.hex())
print("当前搅拌开关为:{}".format("关" if recv_set[3] else '开'))
#获取搅拌开关的工作状态:返回0表示工作状态；返回1表示关闭状态
print('当前加热开关为:{}'.format('关' if recv_set[4] else '开'))
#获取加热开关的工作状态:返回0表示工作状态；返回1表示关闭状态

fda10001010000f403730d
当前搅拌开关为:关
当前加热开关为:关


In [ ]:
#读取仪器状态信息
se.flushInput()
data_state=[0xFE,0XA2,0X00,0X00,0X00,0XA2]
se.write(bytes(data_state))
recv_state=se.read(11)
print(recv_state.hex())
stir_set=recv_state[2]*256+recv_state[3]
stir_now=recv_state[4]*256+recv_state[5]
tem_set=(recv_state[6]*256+recv_state[7])/10
tem_now=(recv_state[8]*256+recv_state[9])/10
print('当前搅拌设置转速为{},当前搅拌转速为{}'.format(stir_set,stir_now))
print('当前搅拌设置温度为{}，当前温度为{}'.format(tem_set,tem_now))
time.sleep(2)#程序暂停2秒

fda2012c000001f4027e44
当前搅拌设置转速为300,当前搅拌转速为0
当前搅拌设置温度为50.0，当前温度为63.8


In [ ]:
#设置设备搅拌转速
se.flushInput()
data_stir=[0xFE,0xB1,0x00,0x00,0x00,0xB1]
stir=300
#设置转速为300转，将转速转为两个十六进制字节结合，第一个字节为转速除以256取整，第二个字节为转速除以256求余，然后重新计算校验
print(f'设置搅拌转速为{stir}')
data_stir[2]=stir//256
data_stir[3]=stir%256
se.write(sum_byts(bytes(data_stir)))
recv_stir=se.read(6)
print(recv_stir.hex())#将字节以十六进制的形式打印出来
if recv_stir[2]==0:
    print("执行正确")
else:
    print("执行错误")
time.sleep(2)


设置搅拌转速为300
fdb1000000b1
执行正确


In [ ]:
#设置设备搅拌温度
se.flushInput()
data_temp=[0xFE,0XB2,0X00,0X00,0X00,0XB2]
temp=50
#设置转速为50度，将温度转为两个十六进制字节结合，第一个字节为转速除以256取整，第二个字节为转速除以256求余，然后重新计算校验
print(f'设置搅拌温度为{temp}')
data_temp[2]=temp*10//256
data_temp[3]=temp*10%256
se.write(sum_byts(bytes(data_temp)))
recv_temp=se.read(6)
if recv_temp[2]==0:
    print("执行正确")
else:
    print("执行错误")
print(recv_temp.hex())#将字节以十六进制的形式打印出来
time.sleep(2)


设置搅拌温度为50
执行正确
fdb2000000b2


In [ ]:
#读取设备开关状态
se.flushInput()
data_set=[0xFE,0XA1,0X00,0X00,0x00,0XA1]
se.write(bytes(data_set))
recv_set=se.read(11)
print(recv_set.hex())
print("当前搅拌开关为:{}".format("关" if recv_set[3] else '开'))
#获取搅拌开关的工作状态:返回0表示工作状态；返回1表示关闭状态
print('当前加热开关为:{}'.format('关' if recv_set[4] else '开'))
#获取加热开关的工作状态:返回0表示工作状态；返回1表示关闭状态
time.sleep(2)

fda10000000000f4027e15
当前搅拌开关为:开
当前加热开关为:开


In [ ]:
#读取设备设置状态
se.flushInput()
data_state=[0xFE,0XA2,0X00,0X00,0X00,0XA2]
se.write(bytes(data_state))
recv_state=se.read(11)
print(recv_state.hex())
stir_set=recv_state[2]*256+recv_state[3]
stir_now=recv_state[4]*256+recv_state[5]
tem_set=(recv_state[6]*256+recv_state[7])/10
tem_now=(recv_state[8]*256+recv_state[9])/10
print('当前搅拌设置转速为{},当前搅拌转速为{}'.format(stir_set,stir_now))
print('当前搅拌设置温度为{}，当前温度为{}'.format(tem_set,tem_now))
time.sleep(2)

fda2012c012301f4027761
当前搅拌设置转速为300,当前搅拌转速为291
当前搅拌设置温度为50.0，当前温度为63.1


In [ ]:
#关闭串口
se.flushInput()
se.close()